In [4]:
from main_classification import ModelWrapper
from models.convformer import Model
import torch

import torchvision
from utils.aug import Resize

In [151]:
p = '/caa/Homes01/mpeer/workspace-git/write/ssl_hisfrag/experiments/papyrow_color_classification_mixconv_20p_train-2023-04-29-21-48/model.pt'
ckpt = torch.load(p)
model = Model(img_size=(128,512))
wrapper = ModelWrapper(model, 25, (128,512))

wrapper.load_state_dict(ckpt['model_state_dict'])

Mixing features of size (512, 4, 16)


<All keys matched successfully>

In [185]:
img = "test_gt.png" #/data/mpeer/papyri_1200/croppedImages/Hermauos_5/Hermauos_5_16.jpg"
test_tf = torchvision.transforms.Compose([
    Resize(512),
    torchvision.transforms.CenterCrop((128,512)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=torch.tensor([0.485, 0.456, 0.406]),
        std=torch.tensor([0.229, 0.224, 0.225])
    )
])

In [186]:
from PIL import Image
im = Image.open(img).convert('RGB')
im.save('inp.png')
m = wrapper.model.backbone
m.model.layer4 = torch.nn.Identity()
m.model.layer3 = torch.nn.Identity()
m.model.layer2 = torch.nn.Identity()
m.model.layer1[2] = torch.nn.Identity()
m.model.layer1[1] = torch.nn.Identity()

In [187]:
inp = test_tf(im).unsqueeze(0)
print(inp.shape)
fm = m(inp)
print(fm.shape)

torch.Size([1, 3, 128, 512])
torch.Size([1, 64, 32, 128])


In [193]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
f = np.sum(fm[0].detach().numpy(), axis=0)
f = fm[0,50].detach().numpy()
print(f.shape)
f = (f - f.min()) / (f.max()- f.min()) 
plt.imshow(f)
plt.show()
cv2.imwrite("test.png", f*255)



(32, 128)


True

In [195]:
plt.tight_layout()
plt.imshow(f, cmap='gray')
plt.axis('off')
plt.show()
plt.savefig("test.jpg",bbox_inches='tight')